In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
            for cell in data['cells']:
                lines = cell['source']
                if type(lines) == str:
                    lines = lines.split('\n')
                line0  = (lines or [''])[0].strip()
                haskey = lambda key: line0 == f'### {key} ###'
                if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                    code = '\n'.join(lines)
                    module.write(code)
                    module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

!cat /proc/meminfo | grep Mem

ipynb = 'ttt-tactics-train'

# %load_ext tensorboard

# !rm -rf "./tensorboard/"


'!!!.ipynb'    ttt-tactics	        ttt_tactics.py
 __pycache__   ttt-tactics.ipynb        ttt-tactics-test.ipynb
 ttt-3.ipynb   ttt-tactics-play.ipynb   ttt-tactics-train.ipynb
INFO:tensorflow:Using local port 15563
INFO:tensorflow:Using local port 17945
INFO:tensorflow:Using local port 24457
INFO:tensorflow:Using local port 23435
INFO:tensorflow:Using local port 15239
INFO:tensorflow:Using local port 23391
INFO:tensorflow:Using local port 19254
INFO:tensorflow:Using local port 22297
INFO:tensorflow:Using local port 23793
INFO:tensorflow:Using local port 19579
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
cuda: False
tensorflow: 2.4.0
python: 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]
module:  ttt_tactics
DIR = ./ttt-tactics
<function savesamples at 0x7f8bcbbcd820>
<function loadsamples at 0x7f8bcbbcda60>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7f8bcbbcdb80>
MemTotal:       16393932 kB
MemFree:        12565748 kB
MemAvailable:  

In [ ]:
minloss = 2.4
model   = AlphaZeroModel()
models  = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])

left = 1000
while left > 0:
    left   -= 1
    path    = f'{DIR}/data-*.tfrec'
    files   = sortedfiles(path)
    dataset = loadsamples(files, buffer_size=200*1024, batch_size=1024, seed=None)
    print('path =',path)
    print('files:',len(files))

    schedule = { 0:0.0001 }
    def onschedule(epoch, lr): 
        if epoch in schedule: return schedule[epoch]  
        else:                 return lr
    scheduler = LearningRateScheduler(onschedule)
    
    @throttle(60*60)
    def onbatch(batch, logs):
        NOTIFY(f'{str(model)}, batch={batch}, logs={logs}', title=ipynb, priority=-1)
    def onepoch(epoch, logs):
        NOTIFY(f'{str(model)}, epoch={epoch}, logs={logs}', title=ipynb, priority=0)
    def ontrain(logs):
        ok = 'OK' if logs['loss'] < minloss else 'NO'
        NOTIFY(f'{str(model)}, train={ok}, logs={logs}', title=ipynb, priority=1)
    notifier = LambdaCallback(
        on_batch_end = onbatch,
        on_epoch_end = onepoch,
        on_train_end = ontrain)
    
    history = model.fit(
        x                   = dataset,
        epochs              = 10,
        verbose             = 1,
        callbacks           = [scheduler,notifier],
        workers             = 10,
        use_multiprocessing = False)
    history = history.history

    loss = history['loss'][-1]
    if loss < minloss:
        minloss = loss
        model.save(nextfile(models[-1]))

    plt.figure()
    plt.plot(history['loss'],        label='total loss')
    plt.plot(history['policy_loss'], label='policy loss')
    plt.plot(history['value_loss'],  label='value loss')
    plt.title('training history')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(loc="upper left")
    plt.savefig(f'{DIR}/{str(model)}.png')
    plt.show()


./ttt-tactics/model-00000009.h5 Oct 18 2020 04:11:03
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
Instructions for updating:
Do not call `graph_parents`.
path = ./ttt-tactics/data-*.tfrec
files: 75
Epoch 1/10
   2523/Unknown - 1820s 717ms/step - loss: 2.3083 - value_loss: 0.7388 - policy_loss: 1.5695 - value_lr: 1.0000e-04

In [ ]:
model.save(f'{DIR}/model-{num:0>3d}.h5')

In [ ]:
|